In [1]:
from skimage.io import imread
import numpy as np
import fnmatch
import os
import re
import random

import tensorflow as tf
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ModelCheckpoint
from keras import backend as K
from keras.backend.tensorflow_backend import set_session
from keras.utils import Sequence, to_categorical

import sys
if "E:\\git\\keras-resnet" not in sys.path:
    sys.path.append("E:\\git\\keras-resnet")
    print(sys.path)
    
import resnet



Using TensorFlow backend.


['', 'C:\\ProgramData\\Anaconda3\\python36.zip', 'C:\\ProgramData\\Anaconda3\\DLLs', 'C:\\ProgramData\\Anaconda3\\lib', 'C:\\ProgramData\\Anaconda3', 'C:\\Users\\hoang\\AppData\\Roaming\\Python\\Python36\\site-packages', 'C:\\ProgramData\\Anaconda3\\lib\\site-packages', 'C:\\ProgramData\\Anaconda3\\lib\\site-packages\\win32', 'C:\\ProgramData\\Anaconda3\\lib\\site-packages\\win32\\lib', 'C:\\ProgramData\\Anaconda3\\lib\\site-packages\\Pythonwin', 'C:\\ProgramData\\Anaconda3\\lib\\site-packages\\IPython\\extensions', 'C:\\Users\\hoang\\.ipython', 'E:\\git\\keras-resnet']


In [ ]:
#TODO check shuffle
class CustomDataGenerator(Sequence):

    def __init__(self, directory, batch_size, classes):
        self.directory = directory
        self.batch_size = batch_size
        self.classes = classes
        #create filenames, labels
        dirs = [d for d in os.listdir(directory) if os.path.isdir(os.path.join(directory, d))]
        print(dirs)
        self.linemap_files = []
        for d in dirs:
            fullpath_dir = os.path.join(directory,d)
            new_elements = [(f,d) for f in os.listdir(fullpath_dir) if self.is_valid_linemap(fullpath_dir, f)]
            self.linemap_files.extend(new_elements)
        
        # shuffle
        random.shuffle(self.linemap_files)
        print(len(self.linemap_files))

        
    def is_valid_linemap(self, input_path, f):  
        if os.path.isfile(os.path.join(input_path, f)) and fnmatch.fnmatch(f, '*linemap.png'):
            rgb_file = f.replace('linemap', 'normal')
            if os.path.exists(os.path.join(input_path,rgb_file)):
                return True
            
        return False
    
    
    def getLabels(self, offsetStr, viewStr): 
        offset = 0
        view = 0
        for i, e in enumerate(self.classes[0]):
            if e == offsetStr:
                offset = i
                break
            
        for i, e in enumerate(self.classes[1]):
            if e == viewStr:
                view = i
                break
            
        return offset, view
    
    
    def __len__(self):
        return int(np.ceil(len(self.linemap_files) / float(self.batch_size)))

    def on_epoch_end(self):
        'Shuffle for the next epoch'
        random.shuffle(self.linemap_files)
    
    def __getitem__(self, idx):
        batch_x = []
        batch_y1 = []
        batch_y2 = []
        batch = self.linemap_files[idx * self.batch_size : (idx + 1) * self.batch_size]
        #print(self.directory)
        #print ('files from getitem(): ', batch)
        p = re.compile('(left|center|right)Offset_(left|straight|right)View')
        for (linemap_file, d) in batch:
            finds = p.findall(d)
            #TODO assert only finds has only 1 element [(_,_)]
            (offsetStr, viewStr) =  finds[0]
            (offset, view) = self.getLabels(offsetStr, viewStr)
            
            folder = os.path.join(self.directory, d)
            rgb_file = linemap_file.replace('linemap','normal')
            rgb = imread(os.path.join(folder, rgb_file))
            
            linemap = imread(os.path.join(folder, linemap_file))[:,:,0]
            linemap = np.reshape(linemap, [512, 512, 1])

            input_array = np.dstack((rgb, linemap)) # 512*512*4
            input_array = input_array / 255.0
            
            batch_x.append(input_array)
            batch_y1.append(offset)
            batch_y2.append(view)
         
        return np.array(batch_x), [np.array(to_categorical(batch_y1, 3)), np.array(to_categorical(batch_y2, 3))]


In [3]:
# #sanity test CustomDataGenerator
# directory = directory = "E:\\UAV_drone\\data\\training\\lateral_and_view_normal_map\\train"
# classes = [['left', 'center', 'right'], ['left', 'straight', 'right']]
# datagen = CustomDataGenerator(directory, 10, classes)
# datagen.__getitem__(0)


In [4]:
config = tf.ConfigProto()
config.gpu_options.per_process_gpu_memory_fraction = 0.6
config.gpu_options.allow_growth = True  #dynamically grow the memory used on the GPU
set_session(tf.Session(config=config))

# dimensions of our images.
img_width, img_height = 512, 512

train_data_dir = 'E:\\UAV_drone\\data\\training\\lateral_and_view_normal_map\\train'
val_data_dir = 'E:\\UAV_drone\\data\\training\\lateral_and_view_normal_map\\val'
nb_train_samples = 52206/2
nb_validation_samples = 15664/2
epochs = 50
batch_size = 16
img_channels = 4
nb_classes = 3

if K.image_data_format() == 'channels_first':
    input_shape = (img_channels, img_width, img_height)
else:
    input_shape = (img_width, img_height, img_channels)

model = resnet.ResnetBuilder.build_resnet_18_uav((img_channels, img_width, img_height), nb_classes, multi_output=2)

model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

# this is the augmentation configuration we will use for training
classes = [['left', 'center', 'right'], ['left', 'straight', 'right']]
train_datagen = CustomDataGenerator(train_data_dir, batch_size, classes)

# this is the augmentation configuration we will use for testing:
val_datagen = CustomDataGenerator(val_data_dir, batch_size, classes)


# checkpoint
current_directory = os.getcwd()
checkpoint_dir = os.path.join(current_directory, 'checkpoint')
if not os.path.exists(checkpoint_dir):
    os.makedirs(checkpoint_dir)

file="model-uav-vertical-{epoch:02d}.hdf5"
checkpoint =  ModelCheckpoint(os.path.join(checkpoint_dir, file), verbose=1)


['centerOffset_leftView', 'centerOffset_rightView', 'centerOffset_straightView', 'leftOffset_leftView', 'leftOffset_rightView', 'leftOffset_straightView', 'rightOffset_leftView', 'rightOffset_rightView', 'rightOffset_straightView']
26103
['centerOffset_leftView', 'centerOffset_rightView', 'centerOffset_straightView', 'leftOffset_leftView', 'leftOffset_rightView', 'leftOffset_straightView', 'rightOffset_leftView', 'rightOffset_rightView', 'rightOffset_straightView']
7832


In [5]:
model.fit_generator(
    train_datagen,
    steps_per_epoch=nb_train_samples // batch_size,
    epochs=epochs,
    validation_data=val_datagen,
    validation_steps=nb_validation_samples // batch_size,
    callbacks=[checkpoint])

Epoch 1/50
1631/1631 [==============================] - 763s 468ms/step - loss: 1.0477 - dense_1_loss: 0.4428 - dense_2_loss: 0.1701 - dense_1_acc: 0.8225 - dense_2_acc: 0.9458 - val_loss: 1.8126 - val_dense_1_loss: 0.8747 - val_dense_2_loss: 0.6651 - val_dense_1_acc: 0.6213 - val_dense_2_acc: 0.7045

Epoch 00001: saving model to C:\Users\hoang\WorkingSpace\TraningModels\keras-resnet\checkpoint\model-uav-vertical-01.hdf5
Epoch 2/50
1631/1631 [==============================] - 774s 475ms/step - loss: 0.6443 - dense_1_loss: 0.3297 - dense_2_loss: 0.0960 - dense_1_acc: 0.8764 - dense_2_acc: 0.9703 - val_loss: 1.7740 - val_dense_1_loss: 0.9749 - val_dense_2_loss: 0.6118 - val_dense_1_acc: 0.7464 - val_dense_2_acc: 0.7896

Epoch 00002: saving model to C:\Users\hoang\WorkingSpace\TraningModels\keras-resnet\checkpoint\model-uav-vertical-02.hdf5
Epoch 3/50
1631/1631 [==============================] - 773s 474ms/step - loss: 0.5549 - dense_1_loss: 0.3042 - dense_2_loss: 0.0827 - dense_1_acc: 0.

1631/1631 [==============================] - 760s 466ms/step - loss: 0.1862 - dense_1_loss: 0.0704 - dense_2_loss: 0.0246 - dense_1_acc: 0.9742 - dense_2_acc: 0.9920 - val_loss: 2.6969 - val_dense_1_loss: 2.5220 - val_dense_2_loss: 0.0830 - val_dense_1_acc: 0.5280 - val_dense_2_acc: 0.9705

Epoch 00039: saving model to C:\Users\hoang\WorkingSpace\TraningModels\keras-resnet\checkpoint\model-uav-vertical-39.hdf5
Epoch 40/50
1631/1631 [==============================] - 761s 466ms/step - loss: 0.1826 - dense_1_loss: 0.0674 - dense_2_loss: 0.0241 - dense_1_acc: 0.9748 - dense_2_acc: 0.9918 - val_loss: 1.6390 - val_dense_1_loss: 1.4731 - val_dense_2_loss: 0.0755 - val_dense_1_acc: 0.6637 - val_dense_2_acc: 0.9770

Epoch 00040: saving model to C:\Users\hoang\WorkingSpace\TraningModels\keras-resnet\checkpoint\model-uav-vertical-40.hdf5
Epoch 41/50
1631/1631 [==============================] - 759s 465ms/step - loss: 0.1872 - dense_1_loss: 0.0720 - dense_2_loss: 0.0237 - dense_1_acc: 0.9732 - de